In [1]:
import requests
from bs4 import BeautifulSoup
import nltk
import gensim.downloader as api
import gensim

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Fetch webpage and parse HTML content
url = 'https://www.flipkart.com/apple-iphone-13-pink-128-gb/p/itm6e30c6ee045d2'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
# Tokenize text into sentences
sentences = []
for paragraph in soup.find_all('p'):
    sentences += paragraph.text.split('.')

In [4]:
words_list = [sentence.split() for sentence in sentences]
print(words_list)

[['APPLE', 'iPhone', '13', '(Pink,', '128', 'GB)'], ['iPhone', '13'], ['boasts', 'an', 'advanced', 'dual-camera', 'system', 'that', 'allows', 'you', 'to', 'click', 'mesmerising', 'pictures', 'with', 'immaculate', 'clarity'], ['Furthermore,', 'the', 'lightning-fast', 'A15', 'Bionic', 'chip', 'allows', 'for', 'seamless', 'multitasking,', 'elevating', 'your', 'performance', 'to', 'a', 'new', 'dimension'], ['A', 'big', 'leap', 'in', 'battery', 'life,', 'a', 'durable', 'design,', 'and', 'a', 'bright', 'Super', 'Retina', 'XDR', 'display', 'facilitate', 'boosting', 'your', 'user', 'experience'], [], ['A', 'more', 'vivid', 'OLED', 'display', 'that’s', 'both', 'easier', 'to', 'see', 'in', 'full', 'sunlight', 'and', 'power', 'efficient'], ['With', 'a', 'durable', 'design', 'that’s', 'water', 'and', 'dust-resistant'], [], ['Automatically', 'create', 'beautiful', 'depth', 'effects', 'and', 'focus', 'transitions', 'in', 'your', 'videos'], ['Bring', 'a', 'new', 'level', 'of', 'storytelling', 'to', '

In [5]:
# Train the word embedding model
model = gensim.models.Word2Vec(words_list, size=100, window=5, min_count=1, workers=4)

In [6]:
# Save the model
model.save('my_word2vec_model.bin')

In [7]:
# Load the model
model = gensim.models.Word2Vec.load('my_word2vec_model.bin')

In [8]:
vector = model.wv['display']  # get numpy vector of a word
sims = model.wv.most_similar('display', topn=5)  # get other similar words

In [9]:
print(sims)

[('facilitate', 0.29216089844703674), ('of', 0.23448902368545532), ('life,', 0.2045225203037262), ('Bangalore', 0.19726967811584473), ('hours', 0.19189152121543884)]


In [ ]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from gensim.similarities.index import AnnoyIndexer

# 100 trees are being used in this example
annoy_index = AnnoyIndexer(model, 100)
# Derive the vector for the word "science" in our model
vector = model.wv["display"]
# The instance of AnnoyIndexer we just created is passed
approximate_neighbors = model.wv.most_similar([vector], topn=5, indexer=annoy_index)
# Neatly print the approximate_neighbors and their corresponding cosine similarity values
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
keyword = 'battery'

matching_sentences = []
if keyword in model.wv:
    keyword_embedding = model.wv[keyword]
    for sentence in sentences:
            tokens = nltk.word_tokenize(sentence)
            # Remove punctuation from the tokens
            tokens = [token for token in tokens if token not in string.punctuation]
            # Remove special characters and numbers from the tokens
            tokens = [token for token in tokens if token.isalpha()]
            # Remove stopwords from the tokens
            stop_words = set(stopwords.words('english'))
            tokens = [token for token in tokens if not token in stop_words]
            sentence_embedding = sum([model.wv[token] for token in tokens]) / len(tokens)
            similarity = model.wv.cosine_similarities(keyword_embedding.reshape(1, -1), sentence_embedding.reshape(1, -1))[0]
            if similarity > 0.5:  # Adjust similarity threshold as needed
                matching_sentences.append(sentence)
                for sentence in matching_sentences:
                  print(sentence)
else:
    print(f"{keyword} not found in model vocabulary.")


In [ ]:
print("Matching sentences:")
print(matching_sentences)